# PyCOMPSs: Integration with Numba

In this example we will how to use Numba with PyCOMPSs.

## Import the PyCOMPSs library

In [ ]:
import pycompss.interactive as ipycompss

## Starting runtime  
Initialize COMPSs runtime
Parameters indicates if the execution will generate task graph, tracefile, monitor interval and debug information.

In [ ]:
ipycompss.start(graph=True, monitor=1000, trace=True)
#debug=True

## Importing task and arguments directionality modules 
Import task module before annotating functions or methods 

In [ ]:
from pycompss.api.task import task
from pycompss.api.parameter import *
from pycompss.api.api import compss_barrier
from pycompss.api.api import compss_wait_on

## Importing other modules
Import the time and numpy modules

In [ ]:
import time
import numpy as np

## Declaring tasks 
Declare functions and decorate with @task those that should be tasks  -- Note that they are exactly the same but the "numba" parameter in the @task decorator

In [ ]:
@task(returns=1)  # Default: numba=False
def ident_loops(x):
    r = np.empty_like(x)
    n = len(x)
    for i in range(n):
        r[i] = np.cos(x[i]) ** 2 + np.sin(x[i]) ** 2
    return r

In [ ]:
@task(returns=1, numba=True)
def ident_loops_jit(x):
    r = np.empty_like(x)
    n = len(x)
    for i in range(n):
        r[i] = np.cos(x[i]) ** 2 + np.sin(x[i]) ** 2
    return r

## Invoking tasks 

In [ ]:
size = 10000000
ntasks = 8

# Run some tasks without numba jit
start = time.time()
for i in range(ntasks):
    out = ident_loops(np.arange(size))
compss_barrier()
end = time.time()

# Run some tasks with numba jit
start_jit = time.time()
for i in range(ntasks):
    out_jit = ident_loops_jit(np.arange(size))
compss_barrier()
end_jit = time.time()

# Get the last result of each run to compare that the results are ok
out = compss_wait_on(out)
out_jit = compss_wait_on(out_jit)

print("TIMING RESULTS:")
print("* ident_loops      : %s seconds" % str(end - start))
print("* ident_loops_jit  : %s seconds" % str(end_jit - start_jit))
if len(out) == len(out_jit) and list(out) == list(out_jit):
    print("* SUCCESS: Results match.")
else:
    print("* FAILURE: Results are different!!!")

## Stop the runtime

In [ ]:
ipycompss.stop(sync=False)

#### <font color=green> **Remember:** *The Python kernel must be restarted before starting again PyCOMPSs within the same notebook.* </font>